In [1]:
import os, os.path
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
import random
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.keras.backend.floatx()
from collections import deque
import itertools
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from libs.utils import *
from libs.generate_boxes import *
from libs.dqn import *

In [32]:
class bbox_Agent(tf.keras.Model):
    def __init__(self, ):
        super(self, bbox_Agent).__init__()
        
        #mbox_list, env state
        #가능하면 다음에 적재할 조합이 state로 들어가면 아주 좋음

SyntaxError: unexpected EOF while parsing (<ipython-input-32-f2f3d8f5ab01>, line 1)

In [2]:
num_episode = 10000
global_step = 0
tr_l, h_fill, tr_r,avg_loss_l,history_eps,used_boxes_eps  = [],[],[],[],[],[]
K = 1
n_candidates = 5
num_max_remain = 180 
num_bbox_type = 20 #사용할 대박스 수
sc_target = 64 #최대 대박스 크기를 sc_target 크기만큼 줄임

In [3]:
mi = 'U1'
mbox_type = 'PP'
f_name = 'plan_predict_input/' + mi+'_'+mbox_type
output_col = ['pack_no','creat_dd', 'odr_no', 'ctmmny_cd','vctp_cd', 'prod_no', 'mbox_cd', 'bbox_cd',
              'B_CBM',  'mbox_acctc_qty', 'bbox_acctc_qty', 
              'packing_rate_volume', 'packing_rate_cbm','x','y','z']

In [4]:
# 대박스 규격 로드
df_bbox = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
casemaster=pd.read_csv('data/preprocessed/casemaster.csv')
df_bbox = df_bbox.iloc[:20]
df_bbox = pd.merge(df_bbox, casemaster[['CD', 'CBM']], how='left', left_on=['CASE_CD'], right_on=['CD']).drop(['CD'],axis=1)
df_bbox = df_bbox.drop(['CASE_CD_count'], axis=1)
df_bbox

,CASE_CD,L,B,H,CBM
0,TBMHW,1190.0,1130.0,500.0,0.672350
1,HBS1FL,1480.0,1130.0,1250.0,2.090500
2,TBS12H,1480.0,1130.0,610.0,1.020160
3,HBS12H,1480.0,1130.0,700.0,1.170680
4,DBM14H,1190.0,1130.0,360.0,0.484092
5,TBS1FLR,1480.0,1130.0,1100.0,1.839640
6,TBM12H,1190.0,1130.0,610.0,0.820267
7,HBS34H,1480.0,1130.0,830.0,1.388092
8,TBM1F,1190.0,1130.0,1100.0,1.479170
9,TBSHW,1480.0,1130.0,500.0,0.836200


In [5]:
# 대박스 규격 로드 (Env)
bbox_size = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
bbox_size = bbox_size.values[:num_bbox_type, 1:-1]
bbox_size = np.concatenate([np.sort(bbox_size[:,:2], axis=1), bbox_size[:,2:]], axis=-1)
bbox_size

array([[1130.0, 1190.0, 500.0],
       [1130.0, 1480.0, 1250.0],
       [1130.0, 1480.0, 610.0],
       [1130.0, 1480.0, 700.0],
       [1130.0, 1190.0, 360.0],
       [1130.0, 1480.0, 1100.0],
       [1130.0, 1190.0, 610.0],
       [1130.0, 1480.0, 830.0],
       [1130.0, 1190.0, 1100.0],
       [1130.0, 1480.0, 500.0],
       [1130.0, 1480.0, 550.0],
       [1130.0, 1190.0, 730.0],
       [1140.0, 1700.0, 1100.0],
       [1130.0, 1480.0, 730.0],
       [1130.0, 1190.0, 550.0],
       [1130.0, 1190.0, 1320.0],
       [1130.0, 1480.0, 1320.0],
       [1480.0, 2260.0, 1100.0],
       [1140.0, 1980.0, 1100.0],
       [1140.0, 1980.0, 610.0]], dtype=object)

In [6]:
# 대박스 스케일링
sc = sc_target/np.max(bbox_size, axis =0)[:2] 
bbox_sc = bbox_size.copy() 
bbox_sc[:, :2] = bbox_size[:,:2] * sc # 세로가로만 스케일링
bbox_sc = (bbox_sc+0.5).astype('int') #반올림
mxl, mxb, mxh = np.max(bbox_sc, axis =0 ) # 최대 규격
print(mxl, mxb, mxh)

64 64 1320


In [7]:
bbox_type = [] #np.zeros((1, 20, 20))
for l,b,h in bbox_sc:
    bbox = np.zeros((mxl, mxb))
    bbox[l:,:] = mxh
    bbox[:,b:] = mxh
    bbox[:l,:b] = mxh - h
    bbox_type.append(bbox)
bbox_type = np.stack((bbox_type)) #(5, 20, 20)
bbox_type

array([[[ 820.,  820.,  820., ..., 1320., 1320., 1320.],
        [ 820.,  820.,  820., ..., 1320., 1320., 1320.],
        [ 820.,  820.,  820., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]],

       [[  70.,   70.,   70., ..., 1320., 1320., 1320.],
        [  70.,   70.,   70., ..., 1320., 1320., 1320.],
        [  70.,   70.,   70., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]],

       [[ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 132

In [43]:
bbox_type.shape

(20, 64, 64)

In [8]:
# 중박스 데이터 로드 
group_key = ['creat_dd', 'odr_no', 'ctmmny_cd', 'mbox_type']
df = pd.read_csv('data/preprocessed/'+f_name +'_resid.csv')
org_col = df.columns
df = df.sort_values('sum_wt', ascending = False)
df = df.set_index(group_key).sort_index()
df

index vctp_cd        prod_no mbox_cd  \
creat_dd odr_no     ctmmny_cd mbox_type                                         
20210104 HDAH21B107 HD        PP             2      AH     4322002505   CM448   
                              PP             0      AH     2451402500   CS558   
                              PP             1      AH     3940202150   CS334   
         HDAH21B111 HD        PP             5      AH     4322002505   CM448   
                              PP             3      AH     2451402500   CS558   
...                                        ...     ...            ...     ...   
20210520 HFHCL45892 HF        PP          6925      HC     71248F9000   HS528   
                              PP          6926      HC  82660J0060SAW   HS334   
         HFSUL45789 HF        PP          6927      SU     52730BW000   HS335   
                              PP          6928      SU     98310BW000   CM216   
                              PP          6929      SU     98700BW000   HS337   

                                         bbox_cd  num_mbox   M_L  M_B  M_H  \
creat_dd odr_no     ctmmny_cd mbox_type                                      
20210104 HDAH21B107 HD        PP          TBSHWH        10   280  265  120   
                              PP          TBSHWH         9   275  210  120   
                              PP          TBS1FW         2   470  350  240   
         HDAH21B111 HD        PP          TBSHWH        17   280  265  120   
                              PP          TBSHWH         1   275  210  120   
...                                          ...       ...   ...  ...  ...   
20210520 HFHCL45892 HF        PP          HBS34H         7   530  280  140   
                              PP          HBS1FW         7   470  350  280   
         HFSUL45789 HF        PP           HBS1F         5   470  350  224   
                              PP         TBM1FLR         4  1060  560  160   
                              PP          HBS1FL        21   470  350  160   

                                         unit_wt   sum_wt  mbox_acctc_qty  \
creat_dd odr_no     ctmmny_cd mbox_type                                     
20210104 HDAH21B107 HD        PP           183.0  14640.0            80.0   
                              PP            19.0  11400.0           600.0   
                              PP           150.0   8100.0            54.0   
         HDAH21B111 HD        PP           183.0  14640.0            80.0   
                              PP            19.0  11400.0           600.0   
...                                          ...      ...             ...   
20210520 HFHCL45892 HF        PP            82.0  12300.0           150.0   
                              PP            99.0   2376.0            24.0   
         HFSUL45789 HF        PP          2985.0  14925.0             5.0   
                              PP           615.0   8610.0            14.0   
                              PP           845.0   5070.0             6.0   

                                         bbox_acctc_qty     B_CBM     B_L  \
creat_dd odr_no     ctmmny_cd mbox_type                                     
20210104 HDAH21B107 HD        PP                   24.0  0.836200  1480.0   
                              PP                   60.0  0.836200  1480.0   
                              PP                   36.0  1.839640  1480.0   
         HDAH21B111 HD        PP                   24.0  0.836200  1480.0   
                              PP                   60.0  0.836200  1480.0   
...                                                 ...       ...     ...   
20210520 HFHCL45892 HF        PP                   50.0  1.388092  1480.0   
                              PP                   36.0  2.090500  1480.0   
         HFSUL45789 HF        PP                   45.0  2.090500  1480.0   
                              PP                   12.0  1.479170  1190.0   
                              PP              

In [9]:
group_idx = df.index.drop_duplicates()
group_idx

MultiIndex([(20210104, 'HDAH21B107', 'HD', 'PP'),
            (20210104, 'HDAH21B111', 'HD', 'PP'),
            (20210104, 'HDAH21B116', 'HD', 'PP'),
            (20210104, 'HFFBKC4417', 'HF', 'PP'),
            (20210104, 'HFFBKC4421', 'HF', 'PP'),
            (20210104, 'HFFBKC4481', 'HF', 'PP'),
            (20210104, 'HFFBKC4485', 'HF', 'PP'),
            (20210104, 'HFFBKC4528', 'HF', 'PP'),
            (20210104, 'HFFBKC4532', 'HF', 'PP'),
            (20210104, 'HFFBKC4546', 'HF', 'PP'),
            ...
            (20210519, 'HFSUL45851', 'HF', 'PP'),
            (20210519, 'KSQLL42358', 'KS', 'PP'),
            (20210519, 'KSQLL52442', 'KS', 'PP'),
            (20210520, 'HDAI21F218', 'HD', 'PP'),
            (20210520, 'HDBI21F202', 'HD', 'PP'),
            (20210520, 'HFFBL45785', 'HF', 'PP'),
            (20210520, 'HFHCL45787', 'HF', 'PP'),
            (20210520, 'HFHCL45808', 'HF', 'PP'),
            (20210520, 'HFHCL45892', 'HF', 'PP'),
            (20210520, 'HFSUL45789

In [10]:
# 중박스 데이터 로드 (입력 데이터)
with open('data/preprocessed/' + f_name +'_resid.pickle', 'rb') as handle: 
    boxes_multi =  pickle.load(handle)
with open('data/preprocessed/' + f_name +'_resid_v0.pickle', 'rb') as handle: 
    boxes_multi_n =  pickle.load(handle)
    
boxes_multi = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi]
boxes_multi_n = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi_n]

# 중박스 스케일링
boxes_multi_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, 2:3]], axis=-1).astype('int') for x in  boxes_multi]
boxes_multi_n_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, [2,5]]], axis=-1).astype('int') for x in  boxes_multi_n]
boxes_multi_sc

[array([[ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 10,   8, 120],
        [ 16,  14, 240],
        [ 16,  14, 240]]),
 array([[ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 12,   8, 120],
        [ 10,   8, 120],
        [ 16,  14, 240]

In [11]:
boxes_multi_n_sc

[array([[ 12,   8, 120,  10],
        [ 10,   8, 120,   9],
        [ 16,  14, 240,   2]]),
 array([[ 12,   8, 120,  17],
        [ 10,   8, 120,   1],
        [ 16,  14, 240,   2]]),
 array([[ 12,   8, 120,  20],
        [ 13,  16, 224,   2],
        [ 10,   8, 120,   3],
        [ 13,  16, 140,  14],
        [ 16,  14, 240,   8],
        [ 10,   8, 120,   2]]),
 array([[ 13,  16, 224,   1],
        [ 23,  20, 224,   2],
        [ 25,  31, 120,   6],
        [ 23,  16, 240,   2],
        [ 16,  14, 280,   4]]),
 array([[ 23,  24, 120,   9],
        [ 25,  31, 120,   6],
        [ 16,  14, 160,  20]]),
 array([[ 23,  20, 224,   6],
        [ 13,  16, 224,   1],
        [ 23,  28, 120,   9],
        [ 25,  31, 120,   3],
        [ 16,  14, 224,   6],
        [ 16,  14, 280,  12]]),
 array([[ 13,  16, 140,   8],
        [ 25,  31, 120,   3],
        [ 23,  24, 120,   3],
        [ 16,  14, 160,  10]]),
 array([[ 16,  14, 160,   1],
        [ 13,  16, 140,  27],
        [ 13,  16, 140,   

In [12]:
cbm_per_mbox = [x[:,-1] for x in boxes_multi]
cbm_per_mbox

[array([0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.01393667, 0.01393667, 0.01393667, 0.01393667, 0.01393667,
        0.01393667, 0.01393667, 0.01393667, 0.01393667, 0.05110111,
        0.05110111]),
 array([0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.01393667, 0.05110111, 0.05110111]),
 array([0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
        0.04181   , 0.04181   , 0.01393667, 0.01393667, 0.01393667,
        0.02776184, 0.02776184, 0.02776184, 0.02776184, 0.02776184,
        0.02776184, 0.02

In [13]:
# 중박스 크기를 행렬로 표현
max_size = sc_target
boxes_mat_multi = np.stack([size2mat(x, num_max_remain, max_size) for x in boxes_multi_n_sc])
boxes_mat_multi

array([[[[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        ...,

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]]],


       [[[120., 120.],
         [120., 120.],
         [120.

In [14]:
# Environment
env = Bpp3DEnvMS(length = mxl, breadth = mxb, height = mxh, bbox_type = bbox_type)
# Agent
agent = DQNAgent( L = mxl, B = mxb , H = mxh, n_remains = num_max_remain, n_loading=K, max_size = max_size,lr=1e-4, exp_steps=9000, 
                 train_st = 1500, memory_len=2000, update_target_rate = 1000, net = 'DDQN_DNN') 

distribution DDQN_DNN


In [15]:
step = 0
e = 0
if e > 9000:
    agent.epsilon = 0
if agent.epsilon > agent.epsilon_end and len(agent.memory)>=agent.train_start:
    agent.epsilon -= agent.epsilon_decay_step
agent.epsilon, agent.epsilon_decay_step

(1.0, 0.00011)

In [20]:
ith_data = e % len(boxes_multi)#5#
boxes_all_sc = boxes_multi_sc[ ith_data].copy().astype('int')# 
cbm_all = cbm_per_mbox[ith_data].copy()
r_boxes = boxes_all_sc.copy()
r_mat = boxes_mat_multi[ith_data].copy() #(50,64,2)
boxes_idx = np.arange(len(boxes_all_sc))
loaded_idx = []
used_boxes, pred_pos = [],[]
env_rewards, is_last, used_bbox = [], [], []
history, h_load, h_remain_size, h_load_size = [],[],[],[]
t_history, t_load, t_remain_size, t_load_size = [],[],[],[]

In [21]:
boxes_all_sc

array([[ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 16,  14, 240],
       [ 16,  14, 240]])

In [22]:
boxes_idx

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [23]:
len(boxes_multi)

1528

In [24]:
r_boxes

array([[ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 10,   8, 120],
       [ 16,  14, 240],
       [ 16,  14, 240]])

In [25]:
done = False
len_group = 0
cbm_reward = 0

In [26]:
# state 선택
in_state = np.array( [ np.concatenate([np.zeros((env.length,env.breadth,2)), b[...,np.newaxis]], axis=-1) for b in bbox_type] )#(5,20,20,3)
in_state = (in_state/env.height).astype(np.float32)
in_state.shape, in_state

((20, 64, 64, 3),
 array([[[[0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          ...,
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ]],
 
         [[0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          ...,
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ]],
 
         [[0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          [0.        , 0.        , 0.6212121 ],
          ...,
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ],
          [0.        , 0.        , 1.        ]],
 
         ...,
 
         [[0.        , 0.        , 1.        ],


In [27]:
loading_loc_c = np.zeros((num_bbox_type, env.length, env.breadth, K)) # 적재 위치

In [33]:
in_state

array([[[[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        [[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        [[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        ...,

        [[0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]

In [34]:
loading_loc_c

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [28]:
loading_loc_c.shape, loading_loc_c

((20, 64, 64, 1),
 array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
   

In [35]:
in_loading = np.zeros((num_bbox_type, K, max_size, 2))
in_loading.shape, in_loading

((20, 1, 64, 2),
 array([[[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]],
 
 
        [[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]],
 
 
        [[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]],
 
 
        ...,
 
 
        [[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]],
 
 
        [[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]],
 
 
        [[[0., 0.],
          [0., 0.],
          [0., 0.],
          ...,
          [0., 0.],
          [0., 0.],
          [0., 0.]]]]))

In [41]:
r_mat

array([[[120., 120.],
        [120., 120.],
        [120., 120.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[120., 120.],
        [120., 120.],
        [120., 120.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[120., 120.],
        [120., 120.],
        [120., 120.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       ...,

       [[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]],

       [[  0.,   0.],
        [  0.,   0.],
        [  0.,   0.],
        ...,
        [  0.,   0.],
        [  0.,   0.],
        [  0.,   0.]]])

In [36]:
in_remain = np.array([r_mat] * num_bbox_type).astype(np.float32)
in_remain.shape, in_remain

((20, 180, 64, 2),
 array([[[[120., 120.],
          [120., 120.],
          [120., 120.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0.]],
 
         [[120., 120.],
          [120., 120.],
          [120., 120.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0.]],
 
         [[120., 120.],
          [120., 120.],
          [120., 120.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0.]],
 
         ...,
 
         [[  0.,   0.],
          [  0.,   0.],
          [  0.,   0.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0.]],
 
         [[  0.,   0.],
          [  0.,   0.],
          [  0.,   0.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0.]],
 
         [[  0.,   0.],
          [  0.,   0.],
          [  0.,   0.],
          ...,
          [  0.,   0.],
          [  0.,   0.],
          [  0.,   0

In [37]:
in_state

array([[[[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        [[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        [[0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         [0.        , 0.        , 0.6212121 ],
         ...,
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]],

        ...,

        [[0.        , 0.        , 1.        ],
         [0.        , 0.        , 1.        ]

In [38]:
loading_loc_c

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [39]:
in_remain

array([[[[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[120., 120.],
         [120., 120.],
         [120., 120.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        ...,

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[  0.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         ...,
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]]],


       [[[120., 120.],
         [120., 120.],
         [120.

In [40]:
in_loading

array([[[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       ...,


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]]])

In [42]:
in_state.shape, loading_loc_c.shape, in_remain.shape, in_loading.shape

((20, 64, 64, 3), (20, 64, 64, 1), (20, 180, 64, 2), (20, 1, 64, 2))

In [31]:
# 대박스 규격 선택
action_idx = agent.get_action(in_state, loading_loc_c, in_remain, in_loading)
action_idx

6

In [55]:
env.reset(action_idx)

In [57]:
history.append(in_state[action_idx])
h_load.append(loading_loc_c[action_idx])
h_remain_size.append(in_remain[action_idx])
h_load_size.append(in_loading[action_idx])

In [58]:
state = env.container_s.copy()
state_h = env.container_h.copy()
global_step += 1
step += 1
state.shape, state_h.shape

((64, 64), (64, 64))

In [59]:
cbm_all_ratio = cbm_all / df_bbox.iloc[env.bbox_idx]['CBM']
cbm_all_ratio

array([0.03549365, 0.03549365, 0.03549365, 0.03549365, 0.03549365,
       0.03549365, 0.03549365, 0.03549365, 0.03549365, 0.03549365,
       0.01419746, 0.01419746, 0.01419746, 0.01419746, 0.01419746,
       0.01419746, 0.01419746, 0.01419746, 0.01419746, 0.05205735,
       0.05205735])

In [60]:
cbm_all

array([0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
       0.03484167, 0.03484167, 0.03484167, 0.03484167, 0.03484167,
       0.01393667, 0.01393667, 0.01393667, 0.01393667, 0.01393667,
       0.01393667, 0.01393667, 0.01393667, 0.01393667, 0.05110111,
       0.05110111])

In [62]:
env.bbox_idx

11

In [61]:
df_bbox

,CASE_CD,L,B,H,CBM
0,TBMHW,1190.0,1130.0,500.0,0.672350
1,HBS1FL,1480.0,1130.0,1250.0,2.090500
2,TBS12H,1480.0,1130.0,610.0,1.020160
3,HBS12H,1480.0,1130.0,700.0,1.170680
4,DBM14H,1190.0,1130.0,360.0,0.484092
5,TBS1FLR,1480.0,1130.0,1100.0,1.839640
6,TBM12H,1190.0,1130.0,610.0,0.820267
7,HBS34H,1480.0,1130.0,830.0,1.388092
8,TBM1F,1190.0,1130.0,1100.0,1.479170
9,TBSHW,1480.0,1130.0,500.0,0.836200


In [44]:
k = min(K, len(r_boxes))
k

1

In [49]:
selected, selected_idx = cbn_select_boxes(r_boxes[:n_candidates],boxes_idx[:n_candidates], k) # 조합
s_order, s_order_idx = get_selected_order(selected, selected_idx, k) #순열

In [50]:
selected, selected_idx

(array([[[ 12,   8, 120]]]), array([[0]]))

In [51]:
s_order, s_order_idx

(array([[[ 12,   8, 120]]]), array([[0]]))

In [52]:
s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전

In [53]:
s_order, s_order_idx

(array([[[ 12,   8, 120]],
 
        [[  8,  12, 120]]]),
 array([[0],
        [0]]))

In [69]:
r_boxes[:n_candidates]

array([[ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120],
       [ 12,   8, 120]])

In [70]:
boxes_idx[:n_candidates]

array([0, 1, 2, 3, 4])

In [71]:
order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
            get_selected_location(s_order, s_order_idx, state, state_h, env.height,K, cbm_reward ,cbm_all_ratio ) #위치

In [72]:
order_idx_c

[0, 1]

In [73]:
num_loading_c

[1, 1]

In [74]:
loading_idx_c

[[0], [0]]

In [75]:
loading_size_c

[[array([ 12,   8, 120])], [array([  8,  12, 120])]]

In [76]:
loading_xyz_c

[[array([  0,   0, 590])], [array([  0,   0, 590])]]

In [77]:
loading_loc_c

[array([[[0.53787879],
         [0.53787879],
         [0.53787879],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.53787879],
         [0.53787879],
         [0.53787879],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.53787879],
         [0.53787879],
         [0.53787879],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],
 
        ...,
 
        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],
 
        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]]),
 array([[[0.53787879],
         [0.53787879],
         

In [78]:
next_state_c

[array([[ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]]),
 array([[ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]])]

In [79]:
next_h_c

[array([[ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]]),
 array([[ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        [ 710.,  710.,  710., ..., 1320., 1320., 1320.],
        ...,
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.],
        [1320., 1320., 1320., ..., 1320., 1320., 1320.]])]

In [80]:
if len(order_idx_c) ==0: #<= 1: #action 스킵
    action_idx = 0
else: # 선택 가능 ->  action 실행
    in_state, in_remain, in_loading = \
        raw2input(state, state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height) # input 변환 
    action_idx = agent.get_action(in_state, np.array(loading_loc_c), in_remain, in_loading)

In [81]:
action_idx

1

In [82]:
len_group

0

In [83]:
# 대박스 선택 후의 target network 입력
if len_group == 0:#len(t_history) == 0: 
    if np.sum(loading_size_c) == 0:#len(order_idx_c) == 0: # 적입 불가능한 경우
        t_history.append(np.stack([state,state_h,env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3)))
        t_load.append(np.zeros((1, env.length, env.breadth, K)) )
        t_remain_size.append(np.zeros((1, num_max_remain, max_size, 2)))
        t_load_size.append(np.zeros((1, K, max_size, 2))) 
        is_last.append(True)
    else: #적입 가능한 경우
        t_history.append([in_state[action_idx]]) # 적입 전 상태
        t_load.append(np.array([loading_loc_c[action_idx]])) # 적재입된 상태
        t_remain_size.append([in_remain[action_idx]]) # 선택 후 전처리된 남은 박스 사이즈
        t_load_size.append([in_loading[action_idx]]) # 적입된 박스 사이즈
        is_last.append(False)

In [84]:
# 다음 step ################################
if np.sum(loading_size_c) == 0: # 주어진 중박스 중 하나도 적재하지 못한 경우
    r_boxes = get_remain(r_boxes[:n_candidates], r_boxes) # 해당 범위의 박스들 제외
    r_mat = r_mat[n_candidates:,:,:]
    r_mat = np.concatenate([r_mat,np.zeros((n_candidates, max_size, 2))])
    boxes_idx = boxes_idx[n_candidates:]
else: # 한 가지 이상의 action 옵션이 있는 경우(적재 가능한 경우)
    #print('loading',loading_idx_c[action_idx] , num_loading_c, np.sum(loading_size_c))
    env.step(next_state_c[action_idx], next_h_c[action_idx])  # env의 다음 state
    # index 업데이트
    order_idx = order_idx_c[action_idx]
    loading_idx = loading_idx_c[action_idx]
    loaded_idx += loading_idx # 적입된 박스 인덱스
    boxes_idx = np.setdiff1d(boxes_idx, s_order_idx[order_idx]) # 남은 박스의 인덱스
    #boxes_idx = np.setdiff1d(boxes_idx, loading_idx) 
    # 남은 박스 업데이트
    r_boxes = boxes_all_sc[boxes_idx]
    r_mat = size2mat(np.concatenate([r_boxes, np.ones((len(r_boxes), 1))], axis = -1).astype('int'), num_max_remain, max_size)
    # cbm
    cbm_reward += (np.sum(cbm_all_ratio[loading_idx]))#/df_bbox.iloc[env.bbox_idx]['CBM'])
    # 사용된 박스, 위치 업데이트
    used_boxes = used_boxes + loading_size_c[action_idx] #사용된 박스 업데이트
    pred_pos = pred_pos + loading_xyz_c[action_idx] #예측된 위치 업데이트
    # bbox
    len_group += num_loading_c[action_idx]

In [85]:
#남은 박스가 없음 or 높이맵이 모두 채워짐 or cbm 모두 채워짐
if len(r_boxes) == 0 or np.sum(env.container_h != env.height) == 0 or cbm_reward>=1:
    #if len(r_boxes) == 0 : print('done: no r_boxes')
    if np.sum(env.container_h != env.height) == 0 : print('done: hegith map > h')
    if cbm_reward>=1: print('done: cbm_reward > 1')
    done = True

In [86]:
# action 선택 수가 2 이상이거나 done일 경우 -> append
if len(num_loading_c) > 1 or done:
    # (s, a) append ########################
    history.append(in_state[action_idx]) # 적재 전 상태
    h_load.append(loading_loc_c[action_idx]) # 적재된 상태
    h_remain_size.append(in_remain[action_idx]) # 선택 후 전처리된 남은 박스 사이즈
    h_load_size.append(in_loading[action_idx]) # 적재된 박스 사이즈
    # (s') append ########################
    t_state = next_state_c[action_idx].copy() # 다음 state 
    t_state_h = next_h_c[action_idx].copy() 
    if done: #마지막 박스까지 채워졌을 경우 or 더 이상 적입할 수 없을 경우
        in_t_history = np.stack([t_state,t_state_h, env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3))
        in_t_history = in_t_history/env.height
        loading_loc_c = np.zeros((1, env.length, env.breadth, K)) 
        in_t_remains = np.zeros((1, num_max_remain, max_size, 2))
        in_t_loading = np.zeros((1, K, max_size, 2))
    else: # 다음 state에 대해 action을 취할 수 있는 경우
        k = min(K, len(r_boxes))
        selected,tmpidx = cbn_select_boxes(r_boxes[:n_candidates], np.arange(len(r_boxes[:n_candidates])), k) # 조합
        s_order,tmpidx = get_selected_order(selected, tmpidx, k) # 순열
        s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
        order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
            get_selected_location(s_order, tmpidx, t_state, t_state_h, env.height, K, cbm_reward ,cbm_all_ratio )
        in_t_history, in_t_remains, in_t_loading = \
            raw2input(t_state, t_state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height)
        if np.sum(loading_loc_c)==0:
            loading_loc_c = np.zeros((1, env.length, env.breadth, K))

    t_history.append(in_t_history) # 적재 전 상태
    t_load.append(np.array(loading_loc_c)) # 적재된 상태
    t_remain_size.append(in_t_remains) # 선택 후 전처리된 남은 박스 사이즈
    t_load_size.append(in_t_loading) # 적재된 박스 사이즈
    # (done) append
    is_last.append(done)

In [87]:
done

False

In [88]:
cbm_reward

0.035493649514600364

In [91]:
is_pred = False
while not done:#중박스에 대해 반복##############
    state = env.container_s.copy()
    state_h = env.container_h.copy()
    global_step += 1
    step += 1
    cbm_all_ratio = cbm_all/df_bbox.iloc[env.bbox_idx]['CBM']
    # 적재 위치 계산 ########################
    k = min(K, len(r_boxes))
    selected, selected_idx = cbn_select_boxes(r_boxes[:n_candidates],boxes_idx[:n_candidates], k) # 조합
    s_order, s_order_idx = get_selected_order(selected, selected_idx, k) #순열
    s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
    order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
        get_selected_location(s_order, s_order_idx, state, state_h, env.height,K, cbm_reward ,cbm_all_ratio ) #위치

    #print('s_order',len(s_order))
    # action ########################
    if len(order_idx_c) ==0: #<= 1: #action 스킵
        action_idx = 0
    else: # 선택 가능 ->  action 실행
        in_state, in_remain, in_loading = \
            raw2input(state, state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height) # input 변환 
        action_idx = agent.get_action(in_state, np.array(loading_loc_c), in_remain, in_loading)

    # 대박스 선택 후의 target network 입력
    if len_group == 0:#len(t_history) == 0: 
        if np.sum(loading_size_c) == 0:#len(order_idx_c) == 0: # 적입 불가능한 경우
            t_history.append(np.stack([state,state_h,env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3)))
            t_load.append(np.zeros((1, env.length, env.breadth, K)) )
            t_remain_size.append(np.zeros((1, num_max_remain, max_size, 2)))
            t_load_size.append(np.zeros((1, K, max_size, 2))) 
            is_last.append(True)
        else: #적입 가능한 경우
            t_history.append([in_state[action_idx]]) # 적입 전 상태
            t_load.append(np.array([loading_loc_c[action_idx]])) # 적재입된 상태
            t_remain_size.append([in_remain[action_idx]]) # 선택 후 전처리된 남은 박스 사이즈
            t_load_size.append([in_loading[action_idx]]) # 적입된 박스 사이즈
            is_last.append(False)

    # 다음 step ################################
    if np.sum(loading_size_c) == 0: # 주어진 중박스 중 하나도 적재하지 못한 경우
        r_boxes = get_remain(r_boxes[:n_candidates], r_boxes) # 해당 범위의 박스들 제외
        r_mat = r_mat[n_candidates:,:,:]
        r_mat = np.concatenate([r_mat,np.zeros((n_candidates, max_size, 2))])
        boxes_idx = boxes_idx[n_candidates:]
    else: # 한 가지 이상의 action 옵션이 있는 경우(적재 가능한 경우)
        #print('loading',loading_idx_c[action_idx] , num_loading_c, np.sum(loading_size_c))
        env.step(next_state_c[action_idx], next_h_c[action_idx])  # env의 다음 state
        # index 업데이트
        order_idx = order_idx_c[action_idx]
        loading_idx = loading_idx_c[action_idx]
        loaded_idx += loading_idx # 적입된 박스 인덱스
        boxes_idx = np.setdiff1d(boxes_idx, s_order_idx[order_idx]) # 남은 박스의 인덱스
        #boxes_idx = np.setdiff1d(boxes_idx, loading_idx) 
        # 남은 박스 업데이트
        r_boxes = boxes_all_sc[boxes_idx]
        r_mat = size2mat(np.concatenate([r_boxes, np.ones((len(r_boxes), 1))], axis = -1).astype('int'), num_max_remain, max_size)
        # cbm
        cbm_reward += (np.sum(cbm_all_ratio[loading_idx]))#/df_bbox.iloc[env.bbox_idx]['CBM'])
        # 사용된 박스, 위치 업데이트
        used_boxes = used_boxes + loading_size_c[action_idx] #사용된 박스 업데이트
        pred_pos = pred_pos + loading_xyz_c[action_idx] #예측된 위치 업데이트
        # bbox
        len_group += num_loading_c[action_idx]


    #남은 박스가 없음 or 높이맵이 모두 채워짐 or cbm 모두 채워짐
    if len(r_boxes) == 0 or np.sum(env.container_h != env.height) == 0 or cbm_reward>=1:
        #if len(r_boxes) == 0 : print('done: no r_boxes')
        if np.sum(env.container_h != env.height) == 0 : print('done: hegith map > h')
        if cbm_reward>=1: print('done: cbm_reward > 1')
        done = True

    # action 선택 수가 2 이상이거나 done일 경우 -> append
    if len(num_loading_c) > 1 or done:
        # (s, a) append ########################
        history.append(in_state[action_idx]) # 적재 전 상태
        h_load.append(loading_loc_c[action_idx]) # 적재된 상태
        h_remain_size.append(in_remain[action_idx]) # 선택 후 전처리된 남은 박스 사이즈
        h_load_size.append(in_loading[action_idx]) # 적재된 박스 사이즈
        # (s') append ########################
        t_state = next_state_c[action_idx].copy() # 다음 state 
        t_state_h = next_h_c[action_idx].copy() 
        if done: #마지막 박스까지 채워졌을 경우 or 더 이상 적입할 수 없을 경우
            in_t_history = np.stack([t_state,t_state_h, env.bbox],axis=-1).reshape((1, env.length, env.breadth, 3))
            in_t_history = in_t_history/env.height
            loading_loc_c = np.zeros((1, env.length, env.breadth, K)) 
            in_t_remains = np.zeros((1, num_max_remain, max_size, 2))
            in_t_loading = np.zeros((1, K, max_size, 2))
        else: # 다음 state에 대해 action을 취할 수 있는 경우
            k = min(K, len(r_boxes))
            selected,tmpidx = cbn_select_boxes(r_boxes[:n_candidates], np.arange(len(r_boxes[:n_candidates])), k) # 조합
            s_order,tmpidx = get_selected_order(selected, tmpidx, k) # 순열
            s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
            order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                get_selected_location(s_order, tmpidx, t_state, t_state_h, env.height, K, cbm_reward ,cbm_all_ratio )
            in_t_history, in_t_remains, in_t_loading = \
                raw2input(t_state, t_state_h, env.bbox, r_mat, num_max_remain, K, s_order[order_idx_c], max_size, env.height)
            if np.sum(loading_loc_c)==0:
                loading_loc_c = np.zeros((1, env.length, env.breadth, K))

        t_history.append(in_t_history) # 적재 전 상태
        t_load.append(np.array(loading_loc_c)) # 적재된 상태
        t_remain_size.append(in_t_remains) # 선택 후 전처리된 남은 박스 사이즈
        t_load_size.append(in_t_loading) # 적재된 박스 사이즈
        # (done) append
        is_last.append(done)

    if done:
        env_rewards.append(cbm_reward)
        #env_rewards.append(env.terminal_reward())
    if cbm_reward>1: break   
    # 학습 시작
    if len(agent.memory) >= agent.train_start & ~is_pred:
        agent.train_model()
        if global_step % agent.update_target_rate == 0: # target model 업데이트
            agent.update_target_model()
        avg_loss_l.append(agent.avg_loss/float(step))